# Exploring GCN

Graph clasification using GCNs implemented in Spektral framework

In [1]:
import pandas as pd
import numpy as np
import matplotlib as mlp
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
import pickle

In [2]:
from sklearn.model_selection import train_test_split
import tensorflow as tf
import spektral as sp

# Restore preprocessed datasets

In [3]:
%store -r np_timeseries

In [4]:
df_patients = pd.read_csv('patients-cleaned.csv', index_col=0)

In [5]:
X = np_timeseries
y = df_patients['target'].values

In [6]:
print(X.shape)
print(y.shape)

(190, 90, 400)
(190,)


In [10]:
# Parameters
samples = X.shape[0]            # Number of subjects.       | 190
N = regions = X.shape[1]        # Number of regions.        | 90
F = X.shape[2]                  # Features - timeseries.    | 400
n_classes = 2                   # Control / patient.        | 2

In [11]:
# gc = np.empty((samples, regions, regions))

# # Granger matrix.
# for p in range(samples):
#     for i in range(regions):
#         ts1 = np_timeseries[p,i,:]
#         for j in range(i, regions):
#             ts2 = np_timeseries[p,j,:]
#             gc_test = sm.tsa.stattools.grangercausalitytests(np.vstack((ts1, ts2)).T,1, verbose=False)
#             p_value = gc_test[1][0]['ssr_ftest'][1]

#             gc[p, i, j] = 0 if p_value < 0.05 else 1

# with open('granger-lambda=1-400-nopreprocessing.pickle', 'wb') as f:
#     pickle.dump(gc, f)

In [64]:
# corr = []
# cov = []

# # Covariance matrix.
# for p in range(samples):
#     ts = np_timeseries[p,:,:]
#     cov.append(np.cov(ts))
#     corr.append(np.corrcoef(ts))

# corr = np.asarray(corr)
# cov = np.asarray(cov)

# with open('covariance-400-nopreprocessing.pickle', 'wb') as f:
#     pickle.dump(cov, f)

# with open('correlation-400-nopreprocessing.pickle', 'wb') as f:
#     pickle.dump(corr, f)

In [69]:
# with open('granger-lambda=1-400-nopreprocessing.pickle', 'rb') as f:
#     gs = pickle.load(f)

# with open('covariance-400-nopreprocessing.pickle', 'rb') as f:
#     cov = pickle.load(f)

with open('correlation-400-nopreprocessing.pickle', 'rb') as f:
    corr = pickle.load(f)

In [72]:
# Threshold graph matrix.
thresh = 0.5

A = np.where(corr < 0.2, 0, 1)

In [73]:
print(f'Graph density: {np.sum(A) / (samples * regions * regions):.2f}')

Graph density: 0.36


## Training and testing datasets

In [92]:
# Train/test split
A_train, A_test, \
x_train, x_test, \
y_train, y_test = train_test_split(A, X, y, test_size=0.2, shuffle=True, random_state=42)


In [96]:
# Generate artificial training data by adding Gaussian noise.
for p in range(x_train.shape[0]):
    print(x_train[p].shape)
    break

(90, 400)


In [83]:
# From `example-spectral`.
l2_reg = 0.0001            # Regularization rate for l2
learning_rate = 0.001     # Learning rate for Adam
epochs = 200              # Number of training epochs
batch_size = 4              # Batch size.

In [88]:
# Model definition
X_in = tf.keras.layers.Input(shape=(N, F))
A_in = tf.keras.layers.Input((N, N))

l2 = tf.keras.regularizers.L2(l2=l2_reg)

gc1 = sp.layers.GraphAttention(8, activation=tf.nn.relu, kernel_regularizer=l2)([X_in, A_in])
gc2 = sp.layers.GraphAttention(8, activation=tf.nn.relu, kernel_regularizer=l2)([gc1, A_in])
pool = sp.layers.GlobalAttentionPool(128)(gc2)

output = tf.keras.layers.Dense(1, activation=tf.nn.sigmoid)(pool)

In [89]:
# Build model
model = tf.keras.models.Model(inputs=[X_in, A_in], outputs=output)
optimizer = tf.keras.optimizers.Adam(lr=learning_rate)
loss = tf.keras.losses.BinaryCrossentropy()

model.compile(optimizer=optimizer, loss=loss, metrics=['acc'])
model.summary()

Model: "functional_20"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_25 (InputLayer)           [(None, 90, 400)]    0                                            
__________________________________________________________________________________________________
input_26 (InputLayer)           [(None, 90, 90)]     0                                            
__________________________________________________________________________________________________
graph_attention_10 (GraphAttent (None, 90, 8)        3224        input_25[0][0]                   
                                                                 input_26[0][0]                   
__________________________________________________________________________________________________
graph_attention_11 (GraphAttent (None, 90, 8)        88          graph_attention_10[0]

In [90]:
#stopping_cb = tf.keras.callbacks.EarlyStopping(patience=es_patience, restore_best_weights=True)
lr_cb = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=50)

In [91]:
# Train model
model.fit(
    [x_train, A_train],
    y_train,
    batch_size=batch_size,
    validation_split=0.1,
    epochs=epochs,
    callbacks=[lr_cb]
)

Epoch 1/200
39/39 [==============================] - 0s 13ms/step - loss: 0.7152 - acc: 0.5817 - val_loss: 0.6524 - val_acc: 0.6667
Epoch 2/200
39/39 [==============================] - 0s 7ms/step - loss: 0.7373 - acc: 0.5359 - val_loss: 0.6877 - val_acc: 0.5556
Epoch 3/200
39/39 [==============================] - 0s 7ms/step - loss: 0.5566 - acc: 0.7059 - val_loss: 0.9185 - val_acc: 0.5000
Epoch 4/200
39/39 [==============================] - 0s 7ms/step - loss: 0.3423 - acc: 0.8627 - val_loss: 0.8841 - val_acc: 0.6111
Epoch 5/200
39/39 [==============================] - 0s 7ms/step - loss: 0.1736 - acc: 0.9412 - val_loss: 1.7173 - val_acc: 0.4444
Epoch 6/200
39/39 [==============================] - 0s 7ms/step - loss: 0.0879 - acc: 0.9739 - val_loss: 2.2196 - val_acc: 0.5000
Epoch 7/200
39/39 [==============================] - 0s 8ms/step - loss: 0.0511 - acc: 0.9739 - val_loss: 1.9094 - val_acc: 0.6667
Epoch 8/200
39/39 [==============================] - 0s 8ms/step - loss: 0.0149 - 

## Only graph no node features

In [36]:
# Parameters
batch_size = 4  # Batch size
epochs = 200  # Number of training epochs
patience = 10  # Patience for early stopping
l2_reg = 0.001  # Regularization rate for l2
l2 = tf.keras.regularizers.L2(l2=l2_reg)

In [37]:
# Model definition
X_in = tf.keras.layers.Input(shape=(N, F))
A_in = tf.keras.layers.Input((N, N))

x = sp.layers.GraphConv(8, activation="elu", kernel_regularizer=l2)([X_in, A_in])
x = sp.layers.GraphConv(8, activation="elu", kernel_regularizer=l2)([x, A_in])
x = sp.layers.GraphConv(8, activation="elu", kernel_regularizer=l2)([x, A_in])

x = sp.layers.GlobalSumPool()(x)
outputs = tf.keras.layers.Dense(1, activation="sigmoid")(x)

In [38]:
model = tf.keras.models.Model(inputs=[X_in, A_in], outputs=outputs)
optimizer = tf.keras.optimizers.Adam(lr=learning_rate)
loss = tf.keras.losses.BinaryCrossentropy()

model.compile(optimizer=optimizer, loss=loss, metrics=['acc'])
model.summary()

Model: "functional_10"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_15 (InputLayer)           [(None, 90, 400)]    0                                            
__________________________________________________________________________________________________
input_16 (InputLayer)           [(None, 90, 90)]     0                                            
__________________________________________________________________________________________________
graph_conv_8 (GraphConv)        (None, 90, 8)        3208        input_15[0][0]                   
                                                                 input_16[0][0]                   
__________________________________________________________________________________________________
graph_conv_9 (GraphConv)        (None, 90, 8)        72          graph_conv_8[0][0]   

In [39]:
# Train model
model.fit(
    [x_train, A_train],
    y_train,
    batch_size=batch_size,
    validation_split=0.1,
    epochs=epochs
)

Epoch 1/200
39/39 [==============================] - 1s 15ms/step - loss: 19970.5059 - acc: 0.5163 - val_loss: 4865.2441 - val_acc: 0.4444
Epoch 2/200
39/39 [==============================] - 0s 4ms/step - loss: 4493.6113 - acc: 0.6078 - val_loss: 5716.4966 - val_acc: 0.2778
Epoch 3/200
39/39 [==============================] - 0s 5ms/step - loss: 1183.4435 - acc: 0.7451 - val_loss: 7570.2681 - val_acc: 0.5556
Epoch 4/200
39/39 [==============================] - 0s 4ms/step - loss: 1352.3500 - acc: 0.8039 - val_loss: 3181.0239 - val_acc: 0.4444
Epoch 5/200
39/39 [==============================] - 0s 5ms/step - loss: 343.5517 - acc: 0.8235 - val_loss: 4501.6235 - val_acc: 0.5000
Epoch 6/200
39/39 [==============================] - 0s 4ms/step - loss: 345.6089 - acc: 0.8301 - val_loss: 4866.2202 - val_acc: 0.4444
Epoch 7/200
39/39 [==============================] - 0s 4ms/step - loss: 510.2189 - acc: 0.7974 - val_loss: 5089.5698 - val_acc: 0.4444
Epoch 8/200
39/39 [=======================